In [46]:
import pandas as pd
import numpy as np
from geopy import distance
import json

In [4]:
camps = pd.concat([pd.read_csv(s, encoding = 'unicode_escape', header=None) for s in ['WestCamp.csv','MidwestCamp.csv','SouthCamp.csv','NortheastCamp.csv','WestCamp.csv','CanadaCamp.csv']])

In [5]:
cities = pd.read_csv('cities.csv')

In [29]:
city_dict = []
dist = 75
for i, c in cities.iterrows():
    loc = (c['lat'],c['lon'])
    min_lat = distance.distance(miles=dist).destination(loc, bearing=180)[0]
    max_lat = distance.distance(miles=dist).destination(loc, bearing=0)[0]
    min_lon = distance.distance(miles=dist).destination(loc, bearing=270)[1]
    max_lon = distance.distance(miles=dist).destination(loc, bearing=90)[1]
    tst = camps.loc[(camps[0]>min_lon)&(camps[0]<max_lon)&(camps[1]>min_lat)&(camps[1]>max_lon)].reset_index(drop=True)
    loc_array = np.array(tst[[1,0]])
    meas = np.array([distance.distance(loc,l).miles for l in loc_array])
    subset = tst.loc[(meas<dist) & (~tst[5].isin(['MIL',' ']))].reset_index(drop=True)
    city_dict.append({'name': c['name'], 'df':subset})

In [43]:
city_dict[2]['df'].to_records()[0]

(0, -88.175, 41.838, 'BLAW/Blackwell Forest Preserve  CP PH:630.933.7248   COMM:Camp Fri Sat nights only - more on holidays SITES:64  AMEN:E DP FT DW SH RS  41.838 -88.175', 'BLAW', 'Blackwell Forest Preserve', 'CP', '630.933.7248 ', ' ', 'Camp Fri Sat nights only - more on holidays', '64', ' ', 'E DP FT DW SH RS ', 'IL', nan, nan, nan)

In [47]:
feature_dict = [
    {
      "type": "Feature",
      "properties": {
        "_umap_options": {
          "iconClass": "Drop",
          "iconUrl": "/uploads/pictogram/campsite-24-white.png",
          "color": "DarkGray"
        },
        "description": "(search for the campground for more info)",
        "name": i[5]
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          i[1],
          i[2]
        ]
      }
    } for i in city_dict[2]['df'].to_records()]

with open("hub_gpx/chicago/chicago_camps.geojson", "w") as outfile:
    json.dump({"type": "FeatureCollection", "features": feature_dict}, outfile, indent=4)